In [3]:
import pandas as pd
import numpy as np
import os
os.listdir('../input')

['train.csv', 'submission_popular.csv', 'item_metadata.csv', 'test.csv']

In [9]:
train = pd.read_csv('../input/train.csv')
train['timestamp'] = pd.to_datetime(train['timestamp'],unit='s')
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,2018-11-01 01:57:40,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,2018-11-01 01:58:42,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,2018-11-01 01:58:42,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,2018-11-01 01:58:52,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,2018-11-01 01:58:52,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [14]:
train = train.sort_values(['user_id','session_id','timestamp']).reset_index(drop = True)
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,0001VQMGUI65,466687f1875ee,2018-11-01 12:38:49,1,interaction item deals,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN
1,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:09,2,interaction item image,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN
2,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:28,3,clickout item,3133074,BR,"Brasília, Brazil",desktop,NaN,3133074|2653226|477811|164667|1990735|104972|1...,43|55|49|51|44|43|47|47|55|46|31|24|33|75|52|4...
3,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:34,4,interaction item image,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN
4,0001VQMGUI65,466687f1875ee,2018-11-01 12:39:44,5,interaction item deals,3133074,BR,"Brasília, Brazil",desktop,NaN,NaN,NaN


clickout을 하든 안 하든 dwelling time을 기준으로 함

In [26]:
reference_first = np.zeros(len(train))
reference_first[0] = 1
check = train['reference'][0]

for i in range(1, len(train)) :
    if check != train['reference'][i] :
        check = train['reference'][i]
        reference_first[i] = 1

In [27]:
reference_first

array([1., 0., 0., ..., 1., 1., 1.])

In [40]:
dwell_df = pd.DataFrame(columns = ['reference','dwell_time'])
dwell_df['reference'] = train[reference_first == 1]['reference']
dwell_df.reset_index(drop = True, inplace = True)

In [49]:
first_time = train[reference_first == 1]['timestamp']
first_time.reset_index(drop = True, inplace = True)

In [50]:
first_time.head()

0   2018-11-01 12:38:49
1   2018-11-01 12:41:14
2   2018-11-01 12:41:16
3   2018-11-01 12:43:47
4   2018-11-01 12:43:57
Name: timestamp, dtype: datetime64[ns]

In [77]:
last_time = pd.Series(np.roll(first_time,-1))

In [90]:
dwell_df['dwell_time'] = (last_time - first_time).astype('timedelta64[s]')

In [100]:
session_end = train[reference_first == 1].groupby(['user_id','session_id']).size().values
session_end

array([11,  4,  5, ..., 13,  1,  9])

In [107]:
session_end = session_end.cumsum()-1

In [112]:
dwell_df = dwell_df.drop(session_end)

In [113]:
dwell_df.head()

,reference,dwell_time
0,3133074,145.0
1,477811,2.0
2,3133074,151.0
3,interaction sort button,10.0
4,price only,0.0


In [115]:
#drop not numeric
dwell_df = dwell_df[dwell_df.reference.apply(lambda x : x.isnumeric())]

In [117]:
#dwell_df.to_csv('../input/dwell_df.csv', index = False)

In [122]:
dwell_df_agg = dwell_df.groupby('reference').agg(['min','max','mean','median'])
dwell_df_agg.head()

dwell_time                           
                 min     max        mean median
reference                                      
100000          80.0    80.0   80.000000   80.0
1000005         89.0  1757.0  923.000000  923.0
1000029          0.0   161.0   27.333333   10.0
10000360       124.0   124.0  124.000000  124.0
1000041          5.0  1264.0  181.354167   76.5

In [126]:
dwell_df_agg = dwell_df_agg.reset_index()
dwell_df_agg.head()

reference dwell_time                           
                   min     max        mean median
0    100000       80.0    80.0   80.000000   80.0
1   1000005       89.0  1757.0  923.000000  923.0
2   1000029        0.0   161.0   27.333333   10.0
3  10000360      124.0   124.0  124.000000  124.0
4   1000041        5.0  1264.0  181.354167   76.5

In [128]:
dwell_df_agg.columns = ['reference','dwell_min','dwell_max','dwell_mean','dwell_median']

In [130]:
dwell_df_agg.to_csv('../input/dwell_time_agg.csv', index = False)